In [1]:

import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 70.0 MB/s eta 0:00:00


In [3]:
# Specify data types for problematic columns
dtype_options = {'nation_position': 'str', 'nation_logo_url': 'str'}

players = pd.read_csv('players_22.csv', dtype=dtype_options)

In [4]:
players.shape

(19239, 110)

In [5]:
players.loc[:, 'pace': 'goalkeeping_speed'].isnull().sum()

pace                            2132
shooting                        2132
passing                         2132
dribbling                       2132
defending                       2132
physic                          2132
attacking_crossing                 0
attacking_finishing                0
attacking_heading_accuracy         0
attacking_short_passing            0
attacking_volleys                  0
skill_dribbling                    0
skill_curve                        0
skill_fk_accuracy                  0
skill_long_passing                 0
skill_ball_control                 0
movement_acceleration              0
movement_sprint_speed              0
movement_agility                   0
movement_reactions                 0
movement_balance                   0
power_shot_power                   0
power_jumping                      0
power_stamina                      0
power_strength                     0
power_long_shots                   0
mentality_aggression               0
m

In [ ]:
#players.dropna(subset=players.loc[:, 'pace': 'goalkeeping_reflexes'].columns, inplace=True)
# We don't drop missing values because some missing values are associated with positions. For example, goalkeepers don't have scores for pace, shooting...

In [6]:
# Separate the position score columns, for example, column 'ls' with value '83+3' separates into column 'ls' with value 83 and column 'ls_potential' 3
for col in players.loc[:, 'ls': 'gk'].columns:
    # Extract the numbers using regular expressions and handle the case where the sign is not present
    extracted_values = players[col].str.extract(r'(\d+)([\+\-]?\d*)')
    # Separate the extracted values into two columns
    players[[col, col+'_potential']] = extracted_values.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # Fill missing values with 0 in the 'Value2' column
    players[col+'_potential'].fillna(0, inplace=True)

In [7]:
players['player_positions'].isnull().sum()

0

In [8]:
# Cleaning the "player_positions" column by removing quotes
players['player_positions'] = players['player_positions'].str.replace('"', '')
players['player_positions'] = players['player_positions'].str.replace(' ', '')

# Splitting the positions and getting dummies
positions_dummies = players['player_positions'].str.get_dummies(sep=',')

# Merging the dummies back into the original dataset
players = pd.concat([players, positions_dummies], axis=1)

In [9]:
# Creating the 'forward_score' column
forward_cols = ['pace', 'shooting', 'passing', 'dribbling', 'physic', 'attacking_crossing',
                'attacking_finishing', 'attacking_heading_accuracy', 'attacking_volleys',
                'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
                'movement_agility', 'movement_reactions', 'power_shot_power', 'power_jumping',
                'power_strength', 'power_long_shots', 'mentality_aggression',
                'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                'mentality_penalties', 'mentality_composure']

players['forward_score'] = players[forward_cols].sum(axis=1)

# Creating the 'mid_score' column
mid_cols = ['pace', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
            'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
            'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_agility',
            'movement_balance', 'mentality_aggression', 'mentality_interceptions',
            'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
            'defending_marking_awareness']

players['mid_score'] = players[mid_cols].sum(axis=1)

# Creating the 'back_score' column
back_cols = ['passing', 'defending', 'physic', 'movement_acceleration', 'movement_sprint_speed',
             'movement_agility', 'movement_reactions', 'movement_balance', 'power_jumping',
             'power_stamina', 'power_strength', 'mentality_positioning', 'mentality_vision',
             'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle',
             'defending_sliding_tackle']

players['back_score'] = players[back_cols].sum(axis=1)

# Creating columns of 'forward', 'midfielder', 'back'
# Forward: CF, ST
# Midfielder: CAM, CDM, CM, LM, LW, LWB, RM, RW, RWB
# Back: CB, LB, RB
# One player can have up to 3 positions
players['forward'] = players[['CF', 'ST']].sum(axis=1)
players.loc[players['forward'] >= 1, 'forward'] = 1

players['midfielder'] = players[['CAM', 'CDM', 'CM', 'LM', 'LW', 'LWB', 'RM', 'RW', 'RWB']].sum(axis=1)
players.loc[players['midfielder'] >= 1, 'midfielder'] = 1

players['back'] = players[['CB', 'LB', 'RB']].sum(axis=1)
players.loc[players['back'] >= 1, 'back'] = 1

In [10]:
# forward_score, mid_score, back_score range from 0-100
from sklearn.preprocessing import MinMaxScaler
features = players.iloc[:, -6:-3] # split numerical features
scaler = MinMaxScaler()
features = scaler.fit_transform(features.values)
players.iloc[:, -6:-3] = features*100

In [ ]:
#players.to_csv('~/Desktop/players.csv', index=False)

In [ ]:
#pd.set_option('display.max_columns', 150)
#players.head()

In [171]:
players_data = players[players['nationality_name'] == 'Italy']
players_data = players_data.fillna(0)
players_data = players_data.sample(300)
players_data.shape

(300, 158)

In [12]:
pd.set_option('display.max_columns', None)
players_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score,forward,midfielder,back
536,192064,https://sofifa.com/player/192064/late-gao/220002,Gao Late,高拉特,"ST,RW,CAM",79,79,17000000.0,32000.0,30,1991-06-05,185,78,111839.0,Guangzhou FC,Chinese Super League,1.0,CAM,11.0,NaN,2020-01-01,2023.0,155,China PR,NaN,NaN,NaN,Right,4,4,2,High/Medium,Normal (185+),No,27200000.0,NaN,"Power Free-Kick, Playmaker (AI), Chip Shot (AI...",83.0,78.0,77.0,79.0,41.0,72.0,73,77,78,81,77,78,70,72,74,83,82,83,74,76,81,78,78,76,73,79,62,33,83,77,76,78,37,41,34,9,14,8,8,7,NaN,79,79,79,79,80,80,80,79,79,79,79,79,75,75,75,79,62,61,61,61,62,59,54,54,54,59,17,https://cdn.sofifa.net/players/192/064/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,86.780715,83.124216,76.923077,1,1,0
538,192598,https://sofifa.com/player/192598/kesen-ai/220002,Ai Kesen,艾克森,ST,79,79,15000000.0,32000.0,31,1989-07-13,180,77,111839.0,Guangzhou FC,Chinese Super League,1.0,RS,9.0,NaN,2019-07-09,2023.0,155,China PR,1413.0,ST,9.0,Right,4,4,2,High/High,Stocky (170-185),No,24000000.0,NaN,"Power Free-Kick, Chip Shot (AI), Technical Dri...",84.0,80.0,73.0,79.0,48.0,79.0,67,80,72,75,76,78,76,80,66,79,87,81,83,75,90,86,83,81,82,72,69,56,77,78,85,76,43,45,35,9,16,7,8,7,NaN,79,79,79,78,78,78,78,78,77,77,77,77,73,73,73,77,65,64,64,64,65,62,59,59,59,62,17,https://cdn.sofifa.net/players/192/598/22_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/teams/1413/60.png,https://cdn.sofifa.net/flags/cn.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,89.683774,86.637390,81.719457,1,0,0
891,188044,https://sofifa.com/player/188044/lan-a/220002,A Lan,阿兰,"ST,LW,LM",77,77,9000000.0,27000.0,31,1989-07-10,178,64,111839.0,Guangzhou FC,Chinese Super League,1.0,SUB,18.0,NaN,2015-12-01,2023.0,155,China PR,1413.0,LW,11.0,Right,3,4,2,High/Low,Lean (170-185),No,14400000.0,NaN,"Long Passer (AI), Speed Dribbl

In [13]:
# Initialize the model
model = gp.Model("soccer_team_optimization")
model.Params.LogToConsole = 0

# Add variables: One for each player, 1 if the player is selected for 23-player team(including substitute), otherwise 0
X = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/23")

# Add variables: One for each player, 1 if the player is selected for 11-player team(excluding substitute), otherwise 0
Y = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/11")

# Add variables: One for each player, 1 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
F = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Forward")
M = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Midfielder")
B = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Back")

# Auxiliary variables for age constraint
Z = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="z")

# Objective 1: Maximize overall rating for 23 players
overall_rating_23 = gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_23, index = 0, priority = 2)

# Objective 2: Maximize skill scores for each position
score_forward = gp.quicksum(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
score_mid = gp.quicksum(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'mid_score'].values[0]
                             for p in players_data['sofifa_id'])
score_back = gp.quicksum(B[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(score_forward+score_mid+score_back, index = 1, priority = 1)

# Objective 3: Maximize overall rating for 11 players
overall_rating_11 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_11, index = 2, priority = 0)

# Objective 4: Maximize team chemistry (number of players from the same club)
chemistry = gp.quicksum(X[p1] * X[p2]
                        for i, p1 in enumerate(players_data['sofifa_id'])
                        for p2 in players_data['sofifa_id'][i+1:]
                        if players_data.loc[players_data['sofifa_id'] == p1, 'club_team_id'].values[0] ==
                           players_data.loc[players_data['sofifa_id'] == p2, 'club_team_id'].values[0])
# model.setObjectiveN(chemistry, index = 3, priority = 0)
## multi objective need to be linear

#model.ModelSense = GRB.MAXIMIZE
model.setObjective(0.4*overall_rating_23+0.3*(score_forward+score_mid+score_back)+0.2*overall_rating_11+0.1*chemistry, GRB.MAXIMIZE)

# Constraint: 23 players including substitutes
model.addConstr(X.sum() == 23)
# Constraint: 11 players excluding substitutes
model.addConstr(Y.sum() == 11)
# Constraint: Relationship between 11 and 23 players
model.addConstrs(Y[p] <= X[p] for p in players_data['sofifa_id'])
# Constraint: for 23 players, each player's age <= 40
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 40
                             for p in players_data['sofifa_id'])
# Constraint: for 23 players, average player's age <= 30, >= 28
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) >= 28*23)
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0]
                            for p in players_data['sofifa_id']) <= 30*23)
# Constraint: for 23 players, at least 3 players with age <= 23 need to be selected
n = len(players_data)
bigM = 100
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'age'].values[0] <= 23+bigM*Z[p]
                             for p in players_data['sofifa_id'])
model.addConstr(Z.sum() <= n-3)
# Constraint: relationship between selected 23 players and their positions
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0] >= F[p]
                             for p in players_data['sofifa_id'])
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0] >= M[p]
                             for p in players_data['sofifa_id'])
model.addConstrs(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0] >= B[p]
                             for p in players_data['sofifa_id'])

# Constraint: one player is selected into the 23-player team for one position, excluding goal keepers
model.addConstrs(F[p]+M[p]+B[p] <= 1 for p in players_data['sofifa_id'])
# Position constraints: number of forwards/midfielders/backs for 23 players
model.addConstr(F.sum() >= 4)
model.addConstr(M.sum() >= 5)
model.addConstr(B.sum() >= 5)
model.addConstr(F.sum() <= 7)
model.addConstr(M.sum() <= 8)
model.addConstr(B.sum() <= 8)
# Position constraint: number of goal keepers
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 3)
# For 23-player team, #CF > =2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CF'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #LW >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] >= p, 'LW'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #RW >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'RW'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #CAM >= 1
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CAM'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 23-player team, #CDM >= 1
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CDM'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 23-player team, #CB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'CB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #LB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'LB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 23-player team, #RB >= 2
model.addConstr(gp.quicksum(X[p] * players_data.loc[players_data['sofifa_id'] == p, 'RB'].values[0]
                            for p in players_data['sofifa_id']) >= 2)
# For 11-player team, 1 GK
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 1)
# For 11-player team, 3-4-3
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0]
                            for p in players_data['sofifa_id']) == 3)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0]
                            for p in players_data['sofifa_id']) == 4)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0]
                            for p in players_data['sofifa_id']) == 3)
# For 11-player team, #LW >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] >= p, 'LW'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #RW >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'RW'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #CB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'CB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #LB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'LB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)
# For 11-player team, #RB >= 1
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'RB'].values[0]
                            for p in players_data['sofifa_id']) >= 1)

# Optimize the model
model.optimize()

# Get the selected players
selected_players23 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if X[p].x > 0]
selected_players11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x > 0]
selected_forwards = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if F[p].x > 0]
selected_midfielders = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if M[p].x > 0]
selected_backs = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if B[p].x > 0]
selected_goalkeepers = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id']
                        if X[p].x * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] > 0]

print("Selected 23 Players:", selected_players23)
print("Selected Forwards:", selected_forwards)
print("Selected Midfielders", selected_midfielders)
print("Selected Backs", selected_backs)
print("Selected Goalkeepers", selected_goalkeepers)
print("Selected 11 Players:", selected_players11)


Restricted license - for non-production use only - expires 2024-10-28


GurobiError: ignored

In [ ]:
players_data['midfielder'].info()

<class 'pandas.core.series.Series'>
Int64Index: 385 entries, 536 to 19234
Series name: midfielder
Non-Null Count  Dtype
--------------  -----
385 non-null    int64
dtypes: int64(1)
memory usage: 6.0 KB


### A little bit of history

One of the best teams because of the way it played was the Netherlands that participated on the WorldCup 1970. Even though it did not won, the way they played was recognized worldwide as "total football". This system was characterized because each player can take over the role of any other player in a team. A player who moves out of his position is replaced by another from his team, thus retaining the team's intended organisational structure. To apply this system, it is important that all the players, except the goalkeeper, have a balanced skills in defensive, middle and forward skills.

From an strategic point of view, the most used formation is 3-2-2-3, with a clearly orientation to attack:

![](https://1.bp.blogspot.com/-bLEF_upRCew/XR5Nu-FXpZI/AAAAAAAADl4/oq2TeBKnn4ALs4d4OBiYER60dZKSktOIACLcBGAs/s1600/holanda%2B1974.jpg)

To apply this, it is important to find the players with the most balanced ratings. Another constraint is that the 4 middlefielders should have at least 84 in skill ball control and at least 84 in passing skills. Additionally, since the total football is oriented to attack, it is important that the side strickers should be very fast, with at least 80 in skill dribling, and the centre forward should have a attacking finishing of at least 80.

In [130]:
# Initialize the model
model = gp.Model("total_football")
model.Params.LogToConsole = 0



# Add variables: One for each player, 1 if the player is selected for 11-player team(excluding substitute), otherwise 0
Y = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/11")

# Add variables: One for each player, 1 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
F = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Forward")
M = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Midfielder")
B = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Back")

# Auxiliary variables for mentality
Z = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="z")

# Objective 1: Maximize overall rating for 23 players
overall_rating_23 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_23, index = 0, priority = 2)

# Objective 2: Maximize skill scores for each position
score_forward = gp.quicksum(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
score_mid = gp.quicksum(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'mid_score'].values[0]
                             for p in players_data['sofifa_id'])
score_back = gp.quicksum(B[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(score_forward+score_mid+score_back, index = 1, priority = 1)

# Objective 3: Maximize overall rating for 11 players
overall_rating_11 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_11, index = 2, priority = 0)


# model.setObjectiveN(chemistry, index = 3, priority = 0)
## multi objective need to be linear

#model.ModelSense = GRB.MAXIMIZE
model.setObjective(0.4*overall_rating_23+0.6*(score_forward+score_mid+score_back), GRB.MAXIMIZE)

# Constraint: 11 players excluding substitutes
model.addConstr(Y.sum() == 11)





# For 11-player team, 3-2-2-3
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0]
                            for p in players_data['sofifa_id']) == 3)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0]
                            for p in players_data['sofifa_id']) == 4)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0]
                            for p in players_data['sofifa_id']) == 3)

# For 11-player team, 1 GK
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 1)

# For 11-player team, #CM or #LM or # RM or #CAM == 4

model.addConstr(gp.quicksum(Y[p] * sum(players_data.loc[players_data['sofifa_id'] == p, ['RM','CM','LM','CAM']].values[0])
                            for p in players_data['sofifa_id']) == 4)

# For 11-player team, skills for midfielder

model.addConstrs(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'passing'].values[0] * players_data.loc[players_data['sofifa_id'] == p,'midfielder'].values[0] >= 84* M[p] for p in players_data['sofifa_id'])
model.addConstrs(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'skill_ball_control'].values[0] * players_data.loc[players_data['sofifa_id'] == p,'midfielder'].values[0] >= 84* M[p] for p in players_data['sofifa_id'])


# For 11-player team, skills for forwards

model.addConstrs(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'dribbling'].values[0] * players_data.loc[players_data['sofifa_id'] == p,'forward'].values[0] >= 80* F[p] for p in players_data['sofifa_id'])

# For 11-player team, striker

model.addConstrs(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'attacking_finishing'].values[0]  >= 80* F[p]  for p in players_data['sofifa_id'])

#model.addConstr(gp.quicksum(Z[p] for p in players_data['sofifa_id']) >= 5)
# Optimize the model
model.optimize()

# Get the selected players

selected_players11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x > 0]
selected_forwards = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if F[p].x > 0]
selected_midfielders = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if M[p].x > 0]
selected_backs = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if B[p].x > 0]
selected_goalkeepers = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id']
                        if Y[p].x * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] > 0]


print("Selected Forwards:", selected_forwards)
print("Selected Midfielders", selected_midfielders)
print("Selected Backs", selected_backs)
print("Selected Goalkeepers", selected_goalkeepers)
print("Selected 11 Players:", selected_players11)


Selected Forwards: ['L. Rodríguez']
Selected Midfielders []
Selected Backs ['L. Vázquez', 'F. Russo', 'S. Ramírez', 'S. Simón', 'L. Martínez Quarta', 'A. Spörle', 'L. Cingolani', 'A. Leguizamón', 'M. Coronel', 'E. Burgos', 'G. Ortíz', 'F. Zapiola', 'T. Cantoro', 'M. Pochettino', 'P. Gazzaniga', 'F. Castet', 'S. Cocimano', 'G. Castellani', 'F. Vera', 'L. Gondou', 'N. Molina', 'L. Giaccone', 'L. Castro', 'I. Gómez', 'J. Álvarez', 'E. Gigliotti', 'F. Pizzicanella', 'B. Valdez', 'F. Parra', 'C. Romero', 'D. Musto', 'G. Tanco', 'E. Godoy', 'J. Rodríguez', 'A. Almendra', 'A. Aguirre', 'I. Machuca', 'I. Scocco', 'T. Martínez', 'E. Piovi', 'N. Figal', 'C. Yacob', 'S. Palacios', 'G. Carrillo', 'A. Fontana', 'K. Zenón', 'D. Juárez', 'E. Olivera', 'F. Torres', 'C. Erbes', 'L. Brochero', 'F. Sánchez', 'S. Barreto', 'M. Romero', 'P. Oro', 'L. Zelarayán', 'J. García', 'S. Rosane', 'C. Parano', 'A. Urzi', 'M. Ramírez', 'F. Pereyra', 'D. Mercado', 'F. Noguera', 'T. Galván', 'B. Sepúlveda', 'L. Ríos', 

On the other side, there is no need to play in an aesthetic way to win. "Catennaccio" was established by Italians in the 50's but it has been applied successfully by many teams. There are many examples but Greece was Euro Champion in 2004 using that style.

From an strategic point of view, the most used formation is 5-3-2, with a clearly orientation to defend, hold and counterattack:

![](https://www.soccer-academy.net/image-files/catenaccio-formation.gif)

To apply this, it is important to find 5 good defenders , with at least 85 score in defending skills. Additionally, the center midfield should be oriented to defensive task as well, with a defending_marking_awareness of at least 80. The goalkeeper is also important, with a goalkeeping_positioning of at least 75. Additionally, the goalkeeper should have a height of at least 1.88 m. Finally, the two strikers should be good at attacking_heading_accuracy with at least 80 as score in that skill, because most of the goals of these kind of teams are from corners or closer free kicks.

In [193]:
# Initialize the model
model = gp.Model("catennacio")
#model.Params.LogToConsole = 0



# Add variables: One for each player, 1 if the player is selected for 11-player team(excluding substitute), otherwise 0
Y = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="player1/11")

# Add variables: One for each player, 1 if the player is selected for 23-player team and is Forward/Midfielder/Back position, otherwise 0
F = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Forward")
M = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Midfielder")
B = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="Back")

# Auxiliary variables for mentality
Z = model.addVars(players_data['sofifa_id'], vtype=GRB.BINARY, name="z")

# Objective 1: Maximize overall rating for 23 players
overall_rating_23 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_23, index = 0, priority = 2)

# Objective 2: Maximize skill scores for each position
score_forward = gp.quicksum(F[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
score_mid = gp.quicksum(M[p] * players_data.loc[players_data['sofifa_id'] == p, 'mid_score'].values[0]
                             for p in players_data['sofifa_id'])
score_back = gp.quicksum(B[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward_score'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(score_forward+score_mid+score_back, index = 1, priority = 1)

# Objective 3: Maximize overall rating for 11 players
overall_rating_11 = gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'overall'].values[0]
                             for p in players_data['sofifa_id'])
#model.setObjectiveN(overall_rating_11, index = 2, priority = 0)


# model.setObjectiveN(chemistry, index = 3, priority = 0)
## multi objective need to be linear

#model.ModelSense = GRB.MAXIMIZE
model.setObjective(0.4*overall_rating_23+0.6*(score_forward+score_mid+score_back), GRB.MAXIMIZE)

# Constraint: 11 players excluding substitutes
model.addConstr(Y.sum() == 11)





# For 11-player team, 5-3-2
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'forward'].values[0]
                            for p in players_data['sofifa_id']) == 2)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'midfielder'].values[0]
                            for p in players_data['sofifa_id']) == 3)
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'back'].values[0]
                            for p in players_data['sofifa_id']) == 5)

# For 11-player team, 1 GK
model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0]
                            for p in players_data['sofifa_id']) == 1)

#for p in players_data['sofifa_id']:
#  model.addConstr(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] * players_data.loc[players_data['sofifa_id']==p,'goalkeeping_positioning'].values[0] >= 75 * Y[p])

#for p in players_data['sofifa_id']:
#  model.addConstr(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] * players_data.loc[players_data['sofifa_id']==p,'height_cm'].values[0] >= 188 * Y[p] )

# For 11-player team,  #CB == 1

model.addConstr(gp.quicksum(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'CB'].values[0]
                           for p in players_data['sofifa_id']) == 1)


for p in players_data['sofifa_id']:
  model.addConstr(Y[p] * players_data.loc[players_data['sofifa_id'] == p,'defending_marking_awareness'].values[0] * players_data.loc[players_data['sofifa_id'] == p,'CB'].values[0] >= 75 * Y[p])


# For 11-player team, skills for backs

#model.addConstrs(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'defending'].values[0] * players_data.loc[players_data['sofifa_id'] == p,'back'].values[0] >= 75* Y[p] for p in players_data['sofifa_id'])


# For 11-player team, striker

#model.addConstrs(Y[p] * players_data.loc[players_data['sofifa_id'] == p, 'attacking_heading_accuracy'].values[0] * players_data.loc[players_data['sofifa_id']==p,'forward'].values[0] >= 75* Y[p]  for p in players_data['sofifa_id'])

#model.addConstr(gp.quicksum(Z[p] for p in players_data['sofifa_id']) >= 5)
# Optimize the model
model.optimize()

# Get the selected players

selected_players11 = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if Y[p].x > 0]
selected_forwards = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if F[p].x > 0]
selected_midfielders = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if M[p].x > 0]
selected_backs = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id'] if B[p].x > 0]
selected_goalkeepers = [players_data.loc[players_data['sofifa_id'] == p, 'short_name'].values[0] for p in players_data['sofifa_id']
                        if Y[p].x * players_data.loc[players_data['sofifa_id'] == p, 'GK'].values[0] > 0]


print("Selected Forwards:", selected_forwards)
print("Selected Midfielders", selected_midfielders)
print("Selected Backs", selected_backs)
print("Selected Goalkeepers", selected_goalkeepers)
print("Selected 11 Players:", selected_players11)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 306 rows, 1500 columns and 1030 nonzeros
Model fingerprint: 0x6c0d25df
Variable types: 0 continuous, 1500 integer (1500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [9e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 300 rows and 1483 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: ignored

In [190]:
players_data[(players_data['goalkeeping_positioning']>=75) & (players_data['GK']==1)]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,ls_potential,st_potential,rs_potential,lw_potential,lf_potential,cf_potential,rf_potential,rw_potential,lam_potential,cam_potential,ram_potential,lm_potential,lcm_potential,cm_potential,rcm_potential,rm_potential,lwb_potential,ldm_potential,cdm_potential,rdm_potential,rwb_potential,lb_potential,lcb_potential,cb_potential,rcb_potential,rb_potential,gk_potential,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,forward_score,mid_score,back_score,forward,midfielder,back
328,205659,https://sofifa.com/player/205659/alessio-cragn...,A. Cragno,Alessio Cragno,GK,81,83,23500000.0,32000.0,27,1994-06-28,184,78,1842.0,Cagliari,Italian Serie A,1.0,GK,28.0,0,2014-07-12,2024.0,27,Italy,0.0,0,0.0,Right,3,1,1,Medium/Medium,Lean (170-185),No,41700000.0,0,Saves with Feet,0.0,0.0,0.0,0.0,0.0,0.0,15,12,12,31,13,13,11,12,25,21,41,37,37,75,43,55,74,43,55,17,33,19,15,42,31,59,20,11,12,82,80,73,80,84,39.0,28,28,28,26,29,29,29,26,29,29,29,28,30,30,30,28,27,29,29,29,27,26,26,26,26,26,80,https://cdn.sofifa.net/players/205/659/22_120.png,https://cdn.sofifa.net/teams/1842/60.png,https://cdn.sofifa.net/flags/it.png,0,https://cdn.sofifa.net/flags/it.png,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,15.292898,12.797992,22.714932,0,0,0
2447,162297,https://sofifa.com/player/162297/daniele-padel...,D. Padelli,Daniele Padelli,GK,73,73,350000.0,7000.0,35,1985-10-25,191,82,55.0,Udinese Calcio,Italian Serie A,1.0,SUB,20.0,0,2021-07-01,2023.0,27,Italy,0.0,0,0.0,Left,2,1,2,Medium/Medium,Normal (185+),No,630000.0,0,Comes For Crosses,0.0,0.0,0.0,0.0,0.0,0.0,11,10,10,26,10,10,13,10,29,21,49,39,38,70,27,47,65,30,45,11,12,13,10,30,22,69,8,12,12,72,72,63,76,70,45.0,25,25,25,24,25,25,25,24,25,25,25,25,26,26,26,25,23,24,24,24,23,23,21,21,21,23,72,https://cdn.sofifa.net/players/162/297/22_120.png,https://cdn.sofifa.net/teams/55/60.png,https://cdn.sofifa.net/flags/it.png,0,https://cdn.sofifa.net/flags/it.png,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,11.145671,7.716437,17.285068,0,0,0
614,225116,https://sofifa.com/player/225116/alex-meret/22...,A. Meret,Alex Meret,GK,79,86,30000000.0,40000.0,24,1997-03-22,190,83,48.0,Napoli,Italian Serie A,1.0,SUB,1.0,0,2018-07-05,2023.0,27,Italy,1343.0,SUB,26.0,Left,3,1,2,Medium/Medium,Lean (185+),No,57000000.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,15,17,19,29,20,16,20,21,22,22,41,48,37,74,48,53,68,29,65,17,22,22,20,21,21,67,10,21,12,80,74,70,78,82,44.0,32,32,32,27,29,29,29,27,27,27,27,28,27,27,27,28,27,27,27,